In [ ]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
# print(data.keys())

In [1]:
# Loading demo from .json
import models
import json

with open("cloud9-vs-outsiders-m1-dust2.json", "r") as file:
# with open("cloud9-vs-outsiders-m2-ancient.json", "r") as file:
    data = json.load(file)

demo = models.deserialize_demo_data(data)

# Creating timeline and events from demo
import timelining

timeline = timelining.create_timeline(demo)
# print(len(timeline.events))
# for event in timeline.events:
#     print(event)

d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in double_scalars
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi
d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in arccos
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi


In [2]:
import timelining

round_boundary_ticks: list[int] = [(round.frames[0].tick, round.frames[-1].tick) for round in demo.game_rounds]

round_frame_ticks: list[list[int]] = []
for round in demo.game_rounds:
    round_frame_ticks.append([frame.tick for frame in round.frames])

organized_events: list[timelining.Event] = [[] for _ in range(0, len(demo.game_rounds))]
for event in timeline.events:
    for frame_index, frame_ticks in enumerate(round_frame_ticks):
        if event.tick in frame_ticks:
            organized_events[frame_index].append(event)
            break
# print(len(organized_events))
# print([len(frame) for frame in organized_events])

import plotting

round = demo.game_rounds[0]

# plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

import cProfile
import pstats
with cProfile.Profile() as pr:
    plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()
stats.dump_stats(filename="info-post-mp-0.prof")

# TODO: Organize events into frames so each frame of the GIF can have the associated events grouped with it

# TODO: Increase frame drawing speed (drawing view vectors is probably what takes the most time so maybe optimize that)
# TODO: Add area controlled calculation for an area graph again

Drawing frames: 100%|██████████| 10/10 [09:09<00:00, 54.95s/it]


         41468495 function calls (40418667 primitive calls) in 550.897 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1636  501.561    0.307  501.561    0.307 {method 'acquire' of '_thread.lock' objects}
      960    2.911    0.003    2.911    0.003 {built-in method _winapi.CreateProcess}
     7491    2.300    0.000    2.300    0.000 {built-in method _winapi.WaitForSingleObject}
       40    1.883    0.047    1.885    0.047 {built-in method matplotlib._image.resample}
       95    1.822    0.019    1.822    0.019 {method 'acquire' of '_multiprocessing.SemLock' objects}
   800868    1.660    0.000    1.660    0.000 {method 'reduce' of 'numpy.ufunc' objects}
1372657/1244957    1.620    0.000    7.125    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   324323    1.574    0.000    1.720    0.000 d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\venv\lib\site-packages\matplo

In [ ]:
# Writing events to csv

import csv
from dataclasses import fields

with open("events.csv", "w") as file:
    writer = csv.writer(file)
    all_fields: list[str] = []
    for event in timeline.events:
        all_fields.extend([field.name for field in fields(event)])
    unique_fields = list(dict.fromkeys(all_fields))
    # print(unique_fields)
    writer.writerow([field for field in unique_fields])
    for event in timeline.events:
        writer.writerow([getattr(event, field, None) for field in unique_fields])

# Writing event descriptions to .csv

import csv
from dataclasses import fields

with open("event_descriptions.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["tick", "name", "description"])
    for event in timeline.events:
        writer.writerow([event.tick, event.__class__.__name__, str(event)])